In [17]:
import math
import numpy as np
import matplotlib.pyplot as plt


In [37]:
class Value:

  # _ op tells us wich operation created this value
  #_children contains konse 2 elements ke beech operation ka result hai yeh no
  def __init__(self,data, _children = (), _op = '') -> None: # like a constructor
    self.data = data
    self._backward = lambda : None # backward is a function which will be called once the entire neural net has been made
    self.grad = 0.0 # this is actually the derivative of final expression with respect to current value (intitally it is zero that means this value has no effect on the final result)
    self._prev = set(_children)
    self._op = _op


  def __repr__(self) -> str: # used to define string representation of an object
    return f"Value(data={self.data})"

  def __add__(self,other):
    out = Value(self.data + other.data,(self,other),'+')

    def _backward(): ## it is a closure function and captures all the local variables that is it will capture the reference of out, self and every other object defined here
      self.grad += 1.0*out.grad
      other.grad += 1.0*out.grad

    out._backward = _backward

    return out

  def __mul__(self,other):
    out = Value(self.data * other.data, (self,other),'*')

    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad

    out._backward = _backward

    return out

  def tanh(self):
    n = self.data
    t = (math.exp(2*n) - 1) / (math.exp(2*n)+1) # this is tanh formula
    print(t)
    out = Value(t,(self,),'tanh')

    def _backward():
      self.grad += (1-t**2) * out.grad
    out._backward = _backward
    return out

  def exp(self):
    x = self.data
    out = Value(math.exp(x),(self,),'exp')

    def _backward():
      self.grad += out.data*out.grad

    out._backward = _backward

  def __truediv__(self,other):
    return self * other**-1

  def __neg__(self):
    return self* Value(-1)

  def __sub__(self,other):
    return self + (-other)


  def __pow__(self,other):
    out = Value(self.data**other,(self,),f"**{other}") # we here assume other is of type integer or float

    def _backward():
      self.grad  += other * (self.data ** (other - 1)) * out.grad

    self._backward = _backward

  def topo(val):
    topo = []
    visited = set()
    que = [val]

    while len(que) > 0:
      size = len(que)
      while size > 0:
        size-=1
        v = que.pop(0) # we can improve the tc by implementing a que
        if v in visited:
          continue
        visited.add(v)
        topo.append(v)
        for child in v._prev:
          que.append(child)
    for node in topo:
      node._backward()




# backpropogation
# we want to find out the derivative of final expression with respect to each value (weights) in the neural nets, this will tell us how each value (weights) influences the final outcome.

# The





In [3]:
import torch
import random

In [2]:
# lets try the same implementation with torch library
# tensors are nothing but 2 d arrays
# torch simplifies the whole process and is much more optimized
x1 = torch.tensor([2.0]).double()
x2 = torch.tensor([0.0]).double()
w1 = torch.tensor([-3.0]).double()
w2 = torch.tensor([1.0]).double()
b = torch.tensor([6.881373]).double()

x1.requires_grad = True
x2.requires_grad = True
w1.requires_grad = True
w2.requires_grad = True


n = x1*w1 + x2*w2 + b

o = torch.tanh(n)
o.backward() # backpropogation

print('x2',x2.grad.item())






x2 0.500000465559325


In [44]:
class Neuron:
  def __init__(self,nin): # nin basically tells us how many inputs comes to this Neuron
    self.w = [Value(random.uniform(-1.0,1.0)) for _ in range(nin)] # for each input we have one corresponding weight
    self.b = Value(random.uniform(-1.0,1.0)) # each neuron will have its own bias

  def __call__(self,x): # this method is called directly from object variables like for ex obj = Neuron, then it will be called as obj(x)
    activation = sum([w*x for w, x in zip(self.w,x)],self.b)
    out = activation.tanh()
    return out

x = [Value(2.0),Value(3.0)]
n = Neuron(2)
print(n(x))




-0.9983278881817537
Value(data=-0.9983278881817537)
